# 🤖 Claude統合研究システム - Colab版

このノートブックを https://colab.research.google.com/drive/13MXbGLL56Li9WYTinOUEoqjkupjycbLr に追加してください。

In [ ]:
# 🚀 Claude研究システム - ワンクリック実行セル
# このセルを既存のColabファイルに追加して実行するだけ！

print("🤖 Claude研究システムを開始します...")
print("=" * 50)

# 1. 環境自動セットアップ
print("🔧 環境をセットアップ中...")
import subprocess
import sys

packages = [
    'torch', 'torchvision', 'transformers', 'ultralytics',
    'opencv-python', 'pillow', 'matplotlib', 'seaborn', 
    'pandas', 'numpy', 'nltk', 'scipy', 'scikit-learn'
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

print("✅ ライブラリインストール完了")

# 2. 必要なライブラリインポート
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration, CLIPProcessor, CLIPModel
from ultralytics import YOLO
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import nltk
import seaborn as sns
from scipy import stats
import os
import warnings
warnings.filterwarnings('ignore')

print("📚 ライブラリインポート完了")

# 3. デバイス設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 使用デバイス: {device}")

# 4. Google Drive接続
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("☁️ Google Drive接続完了")
except:
    print("⚠️ Google Drive接続をスキップ")

print("\n🎯 Claude研究システム準備完了！")
print("次のセルで研究を開始してください。")

In [ ]:
# 🧠 AIモデル自動初期化
print("🤖 AIモデルを初期化中...")

# NLTK準備
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

# BLIP (画像キャプション)
print("📝 BLIP初期化中...")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# CLIP (画像分類)
print("🎯 CLIP初期化中...")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# YOLO (物体検出)
print("🔍 YOLO初期化中...")
yolo_model = YOLO('yolov8n.pt')

print("✅ 全AIモデル初期化完了！")
print("🚀 研究システム準備完了 - 次のセルで研究開始！")

In [ ]:
# 🔬 Claude研究実行システム

class ClaudeResearchSystem:
    def __init__(self):
        self.results = []
        self.semantic_categories = {
            'person': ['person', 'man', 'woman', 'child', 'people', 'human', 'face'],
            'animal': ['dog', 'cat', 'bird', 'horse', 'cow', 'elephant', 'bear', 'lion'],
            'food': ['pizza', 'hamburger', 'cake', 'apple', 'banana', 'sandwich', 'bread'],
            'vehicle': ['car', 'bus', 'truck', 'motorcycle', 'bicycle', 'airplane', 'boat'],
            'building': ['house', 'building', 'bridge', 'castle', 'church', 'tower'],
            'furniture': ['chair', 'table', 'sofa', 'bed', 'desk', 'bench'],
            'plant': ['tree', 'flower', 'grass', 'plant', 'garden', 'forest'],
            'landscape': ['mountain', 'beach', 'lake', 'river', 'sky', 'sunset']
        }
        print("🔬 Claude研究システム初期化完了")
    
    def analyze_image(self, image_path):
        """画像を詳細分析"""
        try:
            # 画像読み込み
            image = Image.open(image_path).convert('RGB')
            print(f"📸 画像読み込み: {image_path}")
            
            # 1. キャプション生成 (BLIP)
            inputs = blip_processor(image, return_tensors="pt").to(device)
            with torch.no_grad():
                out = blip_model.generate(**inputs, max_length=50)
                caption = blip_processor.decode(out[0], skip_special_tokens=True)
            print(f"📝 キャプション: {caption}")
            
            # 2. 意味カテゴリ検出
            detected_category = self.detect_semantic_category(caption)
            print(f"📂 意味カテゴリ: {detected_category}")
            
            # 3. YOLO物体検出
            yolo_results = yolo_model(image_path)
            detected_objects = []
            for r in yolo_results:
                for box in r.boxes:
                    if box.conf[0] > 0.5:  # 確信度50%以上
                        class_id = int(box.cls[0])
                        class_name = yolo_model.names[class_id]
                        confidence = float(box.conf[0])
                        detected_objects.append({
                            'class': class_name, 
                            'confidence': confidence
                        })
            print(f"🔍 検出物体: {len(detected_objects)}個")
            
            # 4. 分類実験 (汎用 vs 特化)
            if detected_category in self.semantic_categories:
                category_labels = self.semantic_categories[detected_category]
                
                # 汎用アプローチ
                general_result = self.classify_with_clip(image, category_labels)
                
                # 特化アプローチ
                specialized_prefix = f"a photo of {detected_category}: "
                specialized_result = self.classify_with_clip(image, category_labels, specialized_prefix)
                
                # 改善率計算
                improvement = (specialized_result['confidence'] - general_result['confidence']) / general_result['confidence'] * 100
                
                print(f"📊 汎用分類: {general_result['label']} ({general_result['confidence']:.3f})")
                print(f"🚀 特化分類: {specialized_result['label']} ({specialized_result['confidence']:.3f})")
                print(f"📈 改善率: {improvement:.2f}%")
            else:
                general_result = specialized_result = {'label': None, 'confidence': None}
                improvement = 0
            
            # 結果まとめ
            result = {
                'image_path': image_path,
                'caption': caption,
                'semantic_category': detected_category,
                'detected_objects_count': len(detected_objects),
                'detected_objects': detected_objects,
                'general_label': general_result['label'],
                'general_confidence': general_result['confidence'],
                'specialized_label': specialized_result['label'],
                'specialized_confidence': specialized_result['confidence'],
                'improvement_rate': improvement
            }
            
            self.results.append(result)
            
            # 画像表示
            plt.figure(figsize=(10, 6))
            plt.imshow(image)
            title = f"{image_path}\n{caption}\nカテゴリ: {detected_category}\n改善率: {improvement:.1f}%"
            plt.title(title, fontsize=12)
            plt.axis('off')
            plt.show()
            
            return result
            
        except Exception as e:
            print(f"❌ 分析エラー: {e}")
            return None
    
    def detect_semantic_category(self, caption):
        """キャプションから意味カテゴリを検出"""
        caption_lower = caption.lower()
        
        category_scores = {}
        for category, keywords in self.semantic_categories.items():
            score = sum(1 for keyword in keywords if keyword in caption_lower)
            if score > 0:
                category_scores[category] = score
        
        if category_scores:
            return max(category_scores, key=category_scores.get)
        return 'general'
    
    def classify_with_clip(self, image, labels, prefix=""):
        """CLIPで画像分類"""
        text_labels = [f"{prefix}{label}" for label in labels]
        inputs = clip_processor(text=text_labels, images=image, return_tensors="pt", padding=True).to(device)
        
        with torch.no_grad():
            outputs = clip_model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1)
            
        max_idx = probs.argmax().item()
        confidence = probs[0][max_idx].item()
        predicted_label = labels[max_idx]
        
        return {'label': predicted_label, 'confidence': confidence}
    
    def generate_research_report(self):
        """研究レポート生成"""
        if not self.results:
            print("❌ 分析結果がありません")
            return
        
        df = pd.DataFrame(self.results)
        
        print("\n" + "=" * 60)
        print("📊 Claude研究システム - 最終レポート")
        print("=" * 60)
        
        # 基本統計
        print(f"\n📈 研究結果サマリー:")
        print(f"  - 分析画像数: {len(df)}")
        print(f"  - 検出カテゴリ数: {df['semantic_category'].nunique()}")
        print(f"  - 総検出物体数: {df['detected_objects_count'].sum()}")
        
        # 改善率分析
        valid_improvements = df[df['improvement_rate'] != 0]['improvement_rate']
        if len(valid_improvements) > 0:
            print(f"\n🚀 特化アプローチ効果:")
            print(f"  - 平均改善率: {valid_improvements.mean():.2f}%")
            print(f"  - 標準偏差: {valid_improvements.std():.2f}%")
            print(f"  - 最大改善率: {valid_improvements.max():.2f}%")
            print(f"  - 最小改善率: {valid_improvements.min():.2f}%")
            
            positive_improvements = valid_improvements[valid_improvements > 0]
            print(f"  - 改善を示した画像: {len(positive_improvements)}/{len(valid_improvements)}")
            
            if len(positive_improvements) > len(valid_improvements) / 2:
                print(f"  🎯 結論: 特化アプローチが効果的です！")
            else:
                print(f"  📝 結論: さらなる改善が必要です")
        
        # カテゴリ別分析
        print(f"\n📊 カテゴリ別詳細:")
        for category in df['semantic_category'].unique():
            cat_data = df[df['semantic_category'] == category]
            cat_improvements = cat_data[cat_data['improvement_rate'] != 0]['improvement_rate']
            print(f"  - {category}: {len(cat_data)}画像", end="")
            if len(cat_improvements) > 0:
                print(f", 平均改善率 {cat_improvements.mean():.2f}%")
            else:
                print(f", 改善率データなし")
        
        # グラフ生成
        self.create_visualizations(df)
        
        # ファイル保存
        self.save_results(df)
        
        return df
    
    def create_visualizations(self, df):
        """可視化グラフ作成"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Claude研究システム - 分析結果', fontsize=16)
        
        # 1. 改善率分布
        valid_improvements = df[df['improvement_rate'] != 0]['improvement_rate']
        if len(valid_improvements) > 0:
            axes[0, 0].hist(valid_improvements, bins=max(5, len(valid_improvements)//2), 
                           alpha=0.7, color='skyblue', edgecolor='black')
            axes[0, 0].axvline(valid_improvements.mean(), color='red', linestyle='--', 
                              label=f'平均: {valid_improvements.mean():.1f}%')
            axes[0, 0].set_title('改善率分布')
            axes[0, 0].set_xlabel('改善率 (%)')
            axes[0, 0].set_ylabel('頻度')
            axes[0, 0].legend()
        
        # 2. カテゴリ別改善率
        category_improvement = df[df['improvement_rate'] != 0].groupby('semantic_category')['improvement_rate'].mean()
        if len(category_improvement) > 0:
            category_improvement.plot(kind='bar', ax=axes[0, 1], color='lightgreen')
            axes[0, 1].set_title('カテゴリ別平均改善率')
            axes[0, 1].set_xlabel('意味カテゴリ')
            axes[0, 1].set_ylabel('平均改善率 (%)')
            axes[0, 1].tick_params(axis='x', rotation=45)
        
        # 3. カテゴリ分布
        category_counts = df['semantic_category'].value_counts()
        axes[1, 0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
        axes[1, 0].set_title('意味カテゴリ分布')
        
        # 4. 確信度比較
        valid_confidences = df[(df['general_confidence'].notna()) & (df['specialized_confidence'].notna())]
        if len(valid_confidences) > 0:
            axes[1, 1].scatter(valid_confidences['general_confidence'], 
                              valid_confidences['specialized_confidence'], alpha=0.7)
            axes[1, 1].plot([0, 1], [0, 1], 'r--', alpha=0.8, label='同等線')
            axes[1, 1].set_title('確信度比較')
            axes[1, 1].set_xlabel('汎用アプローチ確信度')
            axes[1, 1].set_ylabel('特化アプローチ確信度')
            axes[1, 1].legend()
            axes[1, 1].set_xlim(0, 1)
            axes[1, 1].set_ylim(0, 1)
        
        plt.tight_layout()
        plt.show()
    
    def save_results(self, df):
        """結果保存"""
        # CSV保存
        df.to_csv('claude_research_results.csv', index=False, encoding='utf-8')
        
        # JSON保存
        with open('claude_research_results.json', 'w', encoding='utf-8') as f:
            json.dump(self.results, f, ensure_ascii=False, indent=2)
        
        print(f"\n💾 結果保存完了:")
        print(f"  - claude_research_results.csv")
        print(f"  - claude_research_results.json")
        
        # Colabダウンロード
        try:
            from google.colab import files
            files.download('claude_research_results.csv')
            files.download('claude_research_results.json')
            print(f"📥 ファイル自動ダウンロード完了")
        except:
            print(f"ℹ️ ローカル環境で実行中")
        
        # Google Driveバックアップ
        try:
            if os.path.exists('/content/drive/MyDrive'):
                import shutil
                backup_dir = '/content/drive/MyDrive/claude_research_backup'
                os.makedirs(backup_dir, exist_ok=True)
                shutil.copy('claude_research_results.csv', backup_dir)
                shutil.copy('claude_research_results.json', backup_dir)
                print(f"☁️ Google Driveバックアップ完了")
        except:
            pass

# Claude研究システム初期化
claude_research = ClaudeResearchSystem()
print("🎯 Claude研究システム準備完了！")
print("📸 次のセルで画像をアップロードして研究を開始してください。")

In [ ]:
# 🚀 Claude研究実行 - このセルで研究開始！

print("📸 研究用画像をアップロードしてください:")
print("複数の画像を選択可能です（JPG, PNG, BMP など）")
print("=" * 50)

# 画像アップロード
from google.colab import files
uploaded = files.upload()

if uploaded:
    print(f"\n✅ {len(uploaded)}個の画像がアップロードされました")
    print("🔬 Claude研究システムで分析を開始...")
    print("=" * 50)
    
    # 各画像を分析
    for i, filename in enumerate(uploaded.keys()):
        print(f"\n📊 分析中 ({i+1}/{len(uploaded)}): {filename}")
        print("-" * 30)
        
        result = claude_research.analyze_image(filename)
        
        if result:
            print(f"✅ 分析完了")
        else:
            print(f"❌ 分析失敗")
    
    # 研究レポート生成
    print("\n📊 研究レポートを生成中...")
    research_results = claude_research.generate_research_report()
    
    print("\n🎉 Claude研究システム実行完了！")
    print("📥 結果ファイルがダウンロードされました。")
    
else:
    print("❌ 画像がアップロードされませんでした")
    print("📸 上記で画像を選択してからこのセルを再実行してください")

## 🎯 Claude研究システムの特徴

### 🤖 AI技術統合
- **BLIP**: 画像キャプション自動生成
- **CLIP**: 汎用・特化画像分類
- **YOLO**: 高精度物体検出
- **WordNet**: 意味カテゴリ分析

### 📊 自動分析機能
- **意味カテゴリ検出**: 8カテゴリ自動判定
- **改善率計算**: 特化アプローチの効果測定
- **統計分析**: 詳細な研究結果レポート
- **可視化**: 4種類のグラフ自動生成

### 💾 結果出力
- **CSV形式**: 詳細データ
- **JSON形式**: 構造化データ
- **自動ダウンロード**: Colab環境
- **Drive自動バックアップ**: クラウド保存

---
**🚀 このノートブックを既存のColabファイルに追加して使用してください！**